## Module 5 Python Practice


This notebook will reproduce some of the plots created in [L5_Pyramid_Practice](L5_Pyramid_Practice.ipynb) and [L5_Bubble_Practice](L5_Bubble_Practice.ipynb) practice notebooks in Python **without** using the `plotnine` library. 

For the use of `plotnine` in Python for bubble and pyramid charts, refer to the [L5_Bubble_Practice_plotnine](L5_Bubble_Practice_plotnine.ipynb) and [L5_Pyramid_Practice_plotnine](L5_Pyramid_Practice_plotnine.ipynb) notebooks. 

Here, we will use Python's matplotlib library and plotly for replicating bubble charts and population pyramids.

In [ ]:
import pandas as pd
crime = pd.read_csv("/dsa/data/all_datasets/crime.csv")

crime.head()

In [ ]:
list(crime)

The column names have a space in front of them. remove the spaces or update the column names before trying to access them. 

In [ ]:
crime.rename(index=str, columns={" murder": "murder", " Forcible_rate": "Forcible_rate", " Robbery":"Robbery", \
                                " aggravated_assult":"aggravated_assult", " burglary":"burglary", \
                                 " larceny_theft":"larceny_theft", " motor_vehicle_theft":"motor_vehicle_theft", \
                                " population":"population"}, inplace=True)

In [ ]:
list(crime)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# facecolor is just the background color of the plot and figsize controls the size of the plot
fig, ax = plt.subplots(facecolor='w', figsize=(14, 10))

# the bubbles on the plot are plotted based on population of each datapoint. Since the population numbers are huge, 
# radius variable has smaller numbers.  
radius = np.sqrt(crime["population"].astype(float)/np.pi)

# Each iteration will give the row index value and the row in dataframe. 
# Draw the point on the plot and annonate it with state name
for key, row in crime.iterrows():
    ax.scatter(row['murder'], row['burglary'], s=radius[key], alpha=.5)
    ax.annotate(row['state'], xy=(row['murder'], row['burglary']))
plt.show()    

### Let's create the above plot in plotly

In [ ]:
import pandas as pd
import numpy as np

# Plotly in Python 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [ ]:
# The reduced values in radius variable are still big for plotly plot. new_radius has normalized values that can fit 
# in the plot. 
new_radius = radius/50

(**[text and annotations](https://plot.ly/python/text-and-annotations/#custom-text-color-and-styling) : this link has many options you can play with while drawing a bubble chart.**)

The plot has murder on x-axis and burglary on y-axis. The value of mode "markers+text" is drawing the bubbles which are markers, and text is displaying text on these markers. state is given to text argument which is displayed on bubbles. market attribute is used to customize the size and color of bubbles.

In [ ]:


trace = go.Scatter(
    x=crime['murder'],
    y=crime['burglary'],
    mode='markers+text',
    text=crime['state'],
    textfont=dict(
        color='#337711',
        size=10
    ),
        marker = dict(
        size=new_radius,
        color = crime.aggravated_assult,
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace]
iplot(data)

Here's the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html#visualization-scatter) for pandas style of creating scatter plots and bubble plots. 

In [ ]:
from plotnine.data import diamonds
import matplotlib

matplotlib.style.use('ggplot')       # Use ggplot style plots*

In [ ]:
diamonds.head()

### Stacked Bar Chart

Let's try to reproduce the stacked bar chart created in [L5_Pyramid_Practice](L5_Pyramid_Practice.ipynb).

In the R version of ggplot, you can tell the plot to create stacks of bars based on y-axis. You can't do that in plotly. 

R ggplot syntax: 

```R
ggplot(data = diamonds) + geom_bar(mapping = aes(x = cut, fill = clarity))
```

The fill parameter is filling the bars based on frequency distribution of values. But, plotly takes separate input data for y-axis to create stacks of bars. Run the cell below to see how it is done as an example. 

In [ ]:
trace1 = go.Bar(
    x=['giraffes', 'orangutans', 'monkeys'],
    y=[20, 14, 23],
    name='SF Zoo'
)
trace2 = go.Bar(
    x=['giraffes', 'orangutans', 'monkeys'],
    y=[12, 18, 29],
    name='LA Zoo'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

To imitate this behavior, code cell below is creating eight different traces for each category of clarity to produce the stacked bar chart.

In [ ]:
# number of categories in clarity is stored in counts variable. .value_counts() will give levels in a categorical 
# variable. 

counts = len(diamonds.clarity.value_counts())

In [ ]:
# The names of different categories are saved in indexes variable. "Pandas.Series.value_counts().index" will give
# names of the corresponding counrs we are getting from value_counts().

indexes = diamonds.clarity.value_counts().index

Since traces are added to data before plotting, declare data as a list variable. Add traces to data variable in the for loop. Loop is iterated for each category in clarity. A temporary dataframe is created for a category, and cut and clarity columns are assigned to x and y in trace. The trace is appended to data at the end of each iteration.  

In [ ]:
data=[]
for i in range(counts):
    temp_data = diamonds.loc[diamonds.clarity==indexes[i]]
    trace = go.Bar(
        x=temp_data.cut,
        y=temp_data.carat,
        name=indexes[i]
    )
data.append(trace)

In [ ]:
#data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Population Pyramid

In [ ]:
census = pd.read_csv("/dsa/data/all_datasets/census/census2010pyr.csv")
census.head()

Two separate lists are needed with population values for males and females for plotting respective bar charts. The columns are extracted and converted into a list. 

In [ ]:
# subset are created for male and female data rows
male=census[(census.sex == 'Male')]
female=census[(census.sex == 'Female')]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(15, 7))
ax1 = fig.add_subplot(111)

for (i,row) in male.iterrows():
    [barr1, barr2] = plt.bar([i, i], [male['pop'].iloc[i], -female['pop'].iloc[i]], color=['#CC6699', '#008AB8'], width=0.8, align='center', edgecolor='none')
    
plt.xlim(0, 100)
plt.ylim(0, 3e6)
#plt.grid(False, axis='x')
plt.xticks(np.arange(0, 100, 5), fontsize=11,rotation=45)
plt.yticks(np.arange(-3e6, 3e6, 500000))
plt.title("Population by Age and Gender")
plt.xlabel('Age')
plt.ylabel('Population')

plt.legend([barr1, barr2], ["Male", "Female"])

In [ ]:

fig = plt.figure(figsize=(15, 7))
ax1 = fig.add_subplot(111)

# bars are placed SIDE BY SIDE using this [i - 0.2, i + 0.2]

for (i, row) in male.iterrows():
    [barr1, barr2] = plt.bar([i - 0.2, i + 0.2], [male['pop'].iloc[i], female['pop'].iloc[i]], color=['#CC6699', '#008AB8'], width=0.4, align='center', edgecolor='none')
    
plt.xlim(-1, 100)
plt.xticks(range(0, 100, 5), fontsize=11)
plt.grid(False, axis='x')
plt.yticks(np.arange(0, 3e6, 5e5))
plt.title("Population by Age and Gender")
plt.xlabel('Age')
plt.ylabel('Population')

plt.legend([barr1, barr2], ["Male", "Female"])